Team members:

Mohamad Raslan

Ahmad Bassam El Bizri

We will repeat the same analysis we did previously with neural network but now with extra features/attributes from the data which could be related to the control of the smoking of individual such as pressure, close friends, gender, and marital status, etc... to indicate whether they will be relevant or not and whether they improve the performance of the smoke control classification model.

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import pandas as pd

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
personality_individual_characteristics_data = pd.read_csv('personality_individual_characteristics.csv')
display(personality_individual_characteristics_data.head())

,extroversion,critical,dependable,anxious,open,reserved,sympathetic,disorganized,calm,conventional,control,gender,age,stress,employment,exercise,household_size,friend_smokers,age_first
0,Agree strongly,Disagree moderately,Agree strongly,Agree moderately,Agree strongly,Agree a little,Agree strongly,Disagree a little,Agree a little,Disagree strongly,No,Male,28,Frequently,Employed,Never,4,5,17
1,Agree moderately,Agree moderately,Agree moderately,Agree a little,Agree strongly,Disagree strongly,Agree strongly,Neither agree nor disagree,Agree strongly,Neither agree nor disagree,No,Male,20,Occasionally,Unemployed,Often or at least 3 days every week,4,5,14
2,Disagree moderately,Neither agree nor disagree,Agree a little,Agree moderately,Disagree a little,Agree moderately,Neither agree nor disagree,Agree a little,Neither agree nor disagree,Agree moderately,No,Male,24,Constantly,Employed,Every day or at least 5 times every week,4,4,17
3,Agree strongly,Disagree moderately,Agree strongly,Disagree strongly,Agree strongly,Disagree moderately,Agree moderately,Disagree strongly,Neither agree nor disagree,Disagree strongly,No,Male,21,Occasionally,Employed,Often or at least 3 days every week,5,5,16
4,Agree a little,Agree a little,Agree strongly,Disagree moderately,Agree moderately,Agree strongly,Neither agree nor disagree,Disagree moderately,Agree strongly,Disagree strongly,No,Male,21,Never,Employed,Every day or at least 5 times every week,6,1,18


In [3]:
converted_data = personality_individual_characteristics_data.copy()
likert_mapping = {
    "Agree strongly": 3.0,
    "Agree moderately": 2.0,
    "Agree a little": 1.0,
    "Neither agree nor disagree": 0.0,
    "Disagree a little": -1.0,
    "Disagree moderately": -2.0,
    "Disagree strongly": -3.0,
    "": 0.0
}
stress_mapping = {
    "Frequently": 4.0,
    "Occasionally": 3.0,
    "Constantly": 2.0,
    "Rarely": 1.0,
    "Never": 0.0,
    "": 5.0
}
exercise_mapping = {
    "Every day or at least 5 times every week": 3.0,
    "Often or at least 3 days every week": 2.0,
    "Sometimes or a few days every month": 1.0,
    "Never": 0.0  
}
converted_data[['extroversion', 'critical', 'dependable', 'anxious', 'open', 'reserved', 'sympathetic', 'disorganized', 'calm', 'conventional']] = converted_data[['extroversion', 'critical', 'dependable', 'anxious', 'open', 'reserved', 'sympathetic', 'disorganized', 'calm', 'conventional']].map(lambda x: likert_mapping.get(x, x)).fillna(0.0)
converted_data[['stress']] = converted_data[['stress']].map(lambda x: stress_mapping.get(x, x)).fillna(0.0)
converted_data[['exercise']] = converted_data[['exercise']].map(lambda x: exercise_mapping.get(x, x)).fillna(0.0)
converted_data[['control']] = converted_data[['control']].map(lambda x: 0 if x == 'Yes' else 1).fillna(0.0)
converted_data[['gender']] = converted_data[['gender']].map(lambda x: 0 if x == 'Male' else 1).fillna(0.0)
converted_data[['employment']] = converted_data[['employment']].map(lambda x: 1 if x == 'Employed' else 0).fillna(0.0)
control = converted_data['control']
converted_data = converted_data.drop(columns=['control'])
display(converted_data.head())

,extroversion,critical,dependable,anxious,open,reserved,sympathetic,disorganized,calm,conventional,gender,age,stress,employment,exercise,household_size,friend_smokers,age_first
0,3.0,-2.0,3.0,2.0,3.0,1.0,3.0,-1.0,1.0,-3.0,0,28,4.0,1,0.0,4,5,17
1,2.0,2.0,2.0,1.0,3.0,-3.0,3.0,0.0,3.0,0.0,0,20,3.0,0,2.0,4,5,14
2,-2.0,0.0,1.0,2.0,-1.0,2.0,0.0,1.0,0.0,2.0,0,24,2.0,1,3.0,4,4,17
3,3.0,-2.0,3.0,-3.0,3.0,-2.0,2.0,-3.0,0.0,-3.0,0,21,3.0,1,2.0,5,5,16
4,1.0,1.0,3.0,-2.0,2.0,3.0,0.0,-2.0,3.0,-3.0,0,21,0.0,1,3.0,6,1,18


In [4]:
scaler = StandardScaler()
X = scaler.fit_transform(converted_data.values)

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(control.values, dtype=torch.float32)

personality_smoke_tensor_dataset = TensorDataset(X_tensor, y_tensor)

In [5]:
# Split the dataset into train, dev, and test sets
train_size = int(0.75 * len(personality_smoke_tensor_dataset))  # 75% for training
dev_size = int(0.15 * len(personality_smoke_tensor_dataset))  # 10% for validation (dev)
test_size = len(personality_smoke_tensor_dataset) - train_size - dev_size  # Remaining 10% for testing

train_data, dev_data, test_data = random_split(personality_smoke_tensor_dataset, [train_size, dev_size, test_size])

batch_size = 64

# Create DataLoader for train, dev, and test sets
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [134]:
class simpleNN(nn.Module):
    def __init__(self):
        super(simpleNN, self).__init__()
        self.fc1 = nn.Linear(18, 96)  # Input layer
        self.fc2 = nn.Linear(96, 72)  # Hidden layer
        self.fc3 = nn.Linear(72, 36)  # Hidden layer 
        self.fc4 = nn.Linear(36, 1)  # Output layer
        self.dropout = nn.Dropout(0.4)
        self.relu = nn.LeakyReLU()
        self.sigmoid = nn.Sigmoid()
        self.bn1 = nn.BatchNorm1d(96)
        self.bn2 = nn.BatchNorm1d(72)
        self.bn3 = nn.BatchNorm1d(36)
  
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.bn1(x)
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.bn2(x)
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        x = self.bn3(x)
        x = self.dropout(x)
        x = self.sigmoid(self.fc4(x))
        return x

In [135]:
# Instantiate the model, loss function, and optimizer
model = simpleNN()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
epochs = 51

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    running_loss = 0

    for features, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(features) 
        loss = criterion(outputs.squeeze(), labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print loss for the epoch
    if epoch % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')
        

Epoch [1/51], Loss: 0.7669
Epoch [11/51], Loss: 0.6304
Epoch [21/51], Loss: 0.6203
Epoch [31/51], Loss: 0.6048
Epoch [41/51], Loss: 0.5459
Epoch [51/51], Loss: 0.5840


In [136]:
import torch.nn.functional as F

def evaluate_model(loader):
    model.eval()  # Set the model to evaluation mode
    total, correct = 0, 0
    with torch.no_grad():  # Disable gradient calculation
        for features, labels in loader:
            outputs = model(features)
            predicted = (outputs.squeeze() > 0.5).float()            
            total += labels.size(0) 
            correct += (predicted == labels.squeeze()).sum().item()
    return correct / total

accuracy = evaluate_model(train_loader)
print(f'Train Accuracy: {accuracy * 100:.2f}%')
# Evaluate on dev set
accuracy = evaluate_model(dev_loader)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

Train Accuracy: 80.50%
Validation Accuracy: 64.52%


In [137]:
# Evaluate on test set
accuracy = evaluate_model(test_loader)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 77.27%


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6765d258-39ce-4a8b-b9bb-8aeaec5d46e2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>